In [ ]:
#preamble to analysis
import pandas as pd
import numpy as np
from IPython.display import Markdown as md
from IPython.display import HTML, display
from time import sleep

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True,connected=True)
from itables import show as iShow
import itables.options as opt
opt.classes="display compact cell-border"
opt.maxBytes = 0

opt.order = []  # no sorting

In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup
from openFF.common.file_handlers import get_curr_df
from openFF.common.text_handlers import wrap_URL_in_html

In [ ]:
def make_link(row):
    return wrap_URL_in_html(row.link_fn,'link')

# Open-FF Disclosure Index

In [ ]:
df = get_curr_df(cols=['UploadKey','api10','OperatorName','ingKeyPresent','bgStateName','bgCountyName',
                        'TotalBaseWaterVolume','WellName','date'])
c1 = df.bgStateName=='pennsylvania'
c2 = df.bgCountyName.isin(['washington','westmoreland','allegheny'])
gb1 = df[c1&c2].groupby('UploadKey',as_index=False)[['api10','date','OperatorName','bgStateName','bgCountyName','WellName',
                        'TotalBaseWaterVolume']].first()
gb2 = df[c1&c2].groupby('UploadKey',as_index=False)['ingKeyPresent'].any()
mg = pd.merge(gb1,gb2,on='UploadKey',how='left')
mg['apicode'] = mg.api10.str[:5]
mg['link_fn'] = f'disclosures/'+mg.apicode+'/'+mg.UploadKey+'.html'
mg['link'] = mg.apply(make_link,axis=1)

mg = mg[['link','api10','date','OperatorName','bgStateName','bgCountyName',
          'WellName','TotalBaseWaterVolume','ingKeyPresent']].reset_index(drop=True)
mg = mg.rename({'link':'Disclosure','api10':'API Number', 'OperatorName':'Operator Name','bgStateName':'State Name',
               'bgCountyName':'County Name','WellName':'Well Name','TotalBaseWaterVolume':'Water (gal.)',
               'ingKeyPresent':'has chemical records'}, axis=1)
# mg.columns

In [ ]:
iShow(mg,columnDefs=[{"width": "80px", "targets": 2}],classes="display compact cell-border",
     style="table-layout:auto;width:90%;float:center")